In [1]:
# main.py
import torch
from torch_geometric.loader import DataLoader
from src.datasets import MuJoCoPendulumDataset # Changed from FakePendulumDataset
from src.models import DampingGCN
from src.train import run_training, simulate_step, simulate_step_physical # simulate_step might not be needed here if used only in train
from src.config import (
    TOTAL_SAMPLES_FROM_JSON, TRAIN_SPLIT_RATIO, BATCH_SIZE,
    HIDDEN_DIM, NUM_EPOCHS, LEARNING_RATE, WEIGHT_DECAY # Added HIDDEN_DIM
)
import os # For path joining

def main():
    # Step 1: Load dataset (unsupervised mode)
    # Define the root directory for MuJoCo data
    mujoco_data_dir = os.path.join('data', 'mujoco')
    
    # Ensure the processed directory exists for the dataset
    processed_dir = os.path.join(mujoco_data_dir, 'processed')
    os.makedirs(processed_dir, exist_ok=True)

    # You need to ensure your JSON files are in mujoco_data_dir or mujoco_data_dir/raw
    # For this example, assuming they are directly in mujoco_data_dir
    # If they are in data/mujoco/raw, MuJoCoPendulumDataset should find them
    
    # Make sure raw files are "discoverable" by the dataset class
    # E.g., copy them to data/mujoco/raw if they are not already there.
    # For simplicity, I'll assume they are in data/mujoco for now and glob will pick them up
    # Or, ensure the `raw_file_names` and `process` methods correctly locate them.
    # The MuJoCoPendulumDataset is set up to look in self.root first for the pattern.

    print(f"Looking for JSON files in: {mujoco_data_dir}")
    full_dataset = MuJoCoPendulumDataset(root_dir=mujoco_data_dir, json_files_pattern="*.json", mode="unsupervised")
    print(f"Total samples loaded: {len(full_dataset)}")

    num_train = int(TRAIN_SPLIT_RATIO * len(full_dataset))
    num_test = len(full_dataset) - num_train
    
    train_dataset, test_dataset = torch.utils.data.random_split(
        full_dataset, [num_train, num_test], generator=torch.Generator().manual_seed(42)
    )
    print(f"Training samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Step 2: Model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    model = DampingGCN(hidden_dim=HIDDEN_DIM).to(device) # Use HIDDEN_DIM from config

    # Step 3: Train
    run_training(model, train_loader, test_loader, device, 
                 epochs=NUM_EPOCHS, lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    # Optional: Run a test prediction on a sample from the dataset
    if len(full_dataset) > 0:
        sample_idx = 0
        sample = full_dataset[sample_idx].to(device)

        with torch.no_grad():
            model.eval()
            pred_damping = model(sample)
            estimated_b_coeff = pred_damping.squeeze()  # <--- hier hinzugefügt

            sample_dt = sample.dt_step.item()

            pred_next = simulate_step_physical(
                x=sample.x,
                applied_torque=sample.true_torque_t,
                estimated_b=estimated_b_coeff,
                dt=sample_dt,
                mass=sample.true_mass,
                length_com_for_gravity=sample.true_length,
                inertia_yy=sample.inertia_yy,
                gravity_accel=sample.gravity_accel
            )

        print("\nSample prediction (unsupervised):")
        print("Current state θ, ω:")
        print(sample.x.cpu().numpy())
        print("Predicted next state θ_next, ω_next:")
        print(pred_next.cpu().numpy())
        print("True next state θ_next, ω_next (from MuJoCo):")
        print(sample.x_next.cpu().numpy())
        print("Estimated damping per joint (from GNN):")
        print(pred_damping.squeeze().cpu().numpy())
        if hasattr(sample, 'y_true_damping'):
            print("True physical damping per joint (from JSON):")
            print(sample.y_true_damping.cpu().numpy())
        print("Estimated physical damping coeff 'b' (from GNN):")
        print(estimated_b_coeff.cpu().numpy())
        if hasattr(sample, 'y_true_damping_b'):
            print("True physical damping coeff 'b' (from JSON):")
            print(sample.y_true_damping_b.cpu().numpy())
        print(f"  (using mass: {sample.mass.item():.4f}, L_com_gravity: {sample.length_com_for_gravity.item():.4f}, I_yy: {sample.inertia_yy.item():.6e}, dt: {sample.dt_step.item()}, g: {sample.gravity_accel.item()})")

if __name__ == "__main__":
    main()

Looking for JSON files in: data/mujoco
Total samples loaded: 6297
Training samples: 5037, Test samples: 1260
Using device: cpu
Starting training...
Epoch   1 | Train MSE: 23843.6631 | Test MSE: 137.2943
Epoch  20 | Train MSE: 49.4870 | Test MSE: 93.5110
Epoch  40 | Train MSE: 50.0565 | Test MSE: 92.8282
Epoch  60 | Train MSE: 51.1680 | Test MSE: 94.4870
Epoch  80 | Train MSE: 51.8607 | Test MSE: 94.2729
Epoch 100 | Train MSE: 50.8318 | Test MSE: 95.9944
Epoch 120 | Train MSE: 50.9313 | Test MSE: 94.4091
Epoch 140 | Train MSE: 53.4792 | Test MSE: 97.2766
Epoch 160 | Train MSE: 52.2671 | Test MSE: 93.7079
Epoch 180 | Train MSE: 51.0145 | Test MSE: 98.2019
Epoch 200 | Train MSE: 50.8556 | Test MSE: 94.1945
Training done.

Sample prediction (unsupervised):
Current state θ, ω:
[[  1.1645852 116.45853  ]]
Predicted next state θ_next, ω_next:
[[  -4863.6094 -486477.4   ]]
True next state θ_next, ω_next (from MuJoCo):
[[  -5.117799 -628.2384  ]]
Estimated damping per joint (from GNN):
48.45994